In [ ]:
import time

In [ ]:
def folderChecker(my_folder='./my_folder'):
    ## ------- simply clean up the folder path -------
    if my_folder is None:
        my_folder='./tmp'
    elif '/' not in my_folder:
        my_folder = os.path.join(os.getcwd(), my_folder)

    ## ------- Check if the folder exists -------
    check_folder = os.path.isdir(my_folder)
    # os.path.exists(dir_outputs)
    # If the folder does not exist, create it
    if not check_folder:
        os.makedirs(my_folder)
        print(f"\tCreated folder: {my_folder}")
    else:
        print(f'\t{my_folder} is existing')

    return my_folder

##############################################################################################
############################### Loading data from database ###################################
##############################################################################################
def _call_my_query(db_file, my_query):
    import sqlite3
    
    ## connect to the SQLIte database
    my_connection = sqlite3.connect(db_file)

    ## create a cursor object
    my_cursor = my_connection.cursor()

    ## excute the query
    my_cursor.execute(my_query)

    ## fetch all the rows
    rows = my_cursor.fetchall()
    
    # ## export the results
    # data_list = [row for row in rows]
    my_connection.close()
    return rows

## ------------- extract table data from SQLite DB ------------- 
def _extract_tables(db_file, table_name):
    ## get header info
    my_query_colName = f"PRAGMA table_info({table_name})"
    column_details = _call_my_query(db_file, my_query_colName)
    colName_list = [column[1] for column in column_details]

    ## get data info
    my_query_data = f"SELECT * FROM {table_name}"
    data_rows = _call_my_query(db_file, my_query_data)
    
    return colName_list, data_rows

def _write_2_csv(colName_list, data_rows, csv_file_name, delimiter=','):
    import csv
    with open(csv_file_name, 'w', newline='') as csvfh:
        writer = csv.writer(csvfh)    # , delimiter=delimiter
        ## --------- Write header ---------
        writer.writerow(colName_list)

        ## --------- Write data ---------
        print(f"\tNow start writing the data into csv")
        for i in range(0, len(data_rows)):
            writer.writerow(list(data_rows[i]))
            if i % 10**6 == 0:
                print(f"\t\trow-{i}")
    print(f"\tNow the table data were saved into <{csv_file_name}>")
    return None


In [ ]:
################################################################################################
def Step_4_extract_data_from_DB(file_mmpdb, tmp_folder):
    ## count time
    beginTime = time.time()
    print(f"4. Now extracting tables from MMPs database ...")
    ## ------------------------------------------------------------------
    dataDict_csvFiles = {}
    for table_name in ["pair", "compound", "compound_property", "property_name", "constant_smiles",
                    "rule", "rule_smiles", "rule_environment", "rule_environment_statistics", "environment_fingerprint"]:
        
        print(f"\tNow processing the table <{table_name}>")
        colName_list, data_rows = _extract_tables(file_mmpdb, table_name)       

        ## --------- write output ---------
        ## define folder and csv fileName
        subFolderDB = folderChecker(f"{tmp_folder}/DB_tables")
        table_out = f"{subFolderDB}/DB_table_{table_name}.csv"
        ## write 2 csv
        _write_2_csv(colName_list, data_rows, table_out)

        print(f"\t<{table_name}> table has been saved into {table_out}\n")
        dataDict_csvFiles[table_name] = table_out
        # print(table_name)

    ## ------------------------------------------------------------------
    costTime = time.time()-beginTime
    print(f"==> Step 4 <Extracting data from MMPs DB> complete, costs time = %ds ................\n" % (costTime))    
    return dataDict_csvFiles